<a href="https://colab.research.google.com/github/uditmanav17/DL-with-Python/blob/master/Ch_7_2_Keras_callbacks_and_TensorBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##7.2 Inspecting and monitoring deep-learning models using Keras callbacks and TensorBoard
In this section, we’ll review ways to gain greater access to and control over what goes on inside your model during training. Launching a training run on a large dataset for tens of epochs using model.fit() or model.fit_generator() can be a bit like launching a paper airplane: past the initial impulse, you don’t have any control over its trajectory or its landing spot. If you want to avoid bad outcomes (and thus wasted paper airplanes), it’s smarter to use not a paper plane, but a drone that can sense its environment, send data back to its operator, and automatically make steering decisions based on its current state. The techniques we present here will transform the call to model.fit() from a paper airplane into a smart, autonomous drone that can selfintrospect and dynamically take action. 

###7.2.1 Using callbacks to act on a model during training
When you’re training a model, there are many things you can’t predict from the start. In particular, you can’t tell how many epochs will be needed to get to an optimal validation loss. The examples so far have adopted the strategy of training for enough epochs that you begin overfitting, using the first run to figure out the proper number of epochs to train for, and then finally launching a new training run from scratch using this optimal number. Of course, this approach is wasteful.

A much better way to handle this is to stop training when you measure that the validation loss in no longer improving. This can be achieved using a Keras callback. A callback is an object (a class instance implementing specific methods) that is passed to the model in the call to fit and that is called by the model at various points during training. It has access to all the available data about the state of the model and its performance, and it can take action: interrupt training, save a model, load a different weight set, or otherwise alter the state of the model.

Here are some examples of ways you can use callbacks:
* Model checkpointing—Saving the current weights of the model at different points during training.
* Early stopping—Interrupting training when the validation loss is no longer improving (and of course, saving the best model obtained during training).
* Dynamically adjusting the value of certain parameters during training—Such as the learning rate of the optimizer.
* Logging training and validation metrics during training, or visualizing the representations learned by the model as they’re updated—The Keras progress bar that you’re familiar with is a callback!

The keras.callbacks module includes a number of built-in callbacks (this is not an exhaustive list):
```
keras.callbacks.ModelCheckpoint
keras.callbacks.EarlyStopping
keras.callbacks.LearningRateScheduler
keras.callbacks.ReduceLROnPlateau
keras.callbacks.CSVLogger
```
Let’s review a few of them to give you an idea of how to use them: ModelCheckpoint, EarlyStopping, and ReduceLROnPlateau.


#### THE MODELCHECKPOINT AND EARLYSTOPPING CALLBACKS
You can use the EarlyStopping callback to interrupt training once a target metric being monitored has stopped improving for a fixed number of epochs. For instance, this callback allows you to interrupt training as soon as you start overfitting, thus avoiding having to retrain your model for a smaller number of epochs. This callback is typically used in combination with ModelCheckpoint, which lets you continually save the model during training (and, optionally, save only the current best model so far: the version of the model that achieved the best performance at the end of an epoch):

In [0]:
import keras

# Callbacks are passed to the model via the
# callbacks argument in fit, which takes a list of
# callbacks. You can pass any number of callbacks.
callbacks_list = [
                  # Interrupts training when improvement stops
                  keras.callbacks.EarlyStopping( 
                      # Monitors the model’s validation accuracy
                      monitor='acc', 
                      # Interrupts training when accuracy has stopped improving 
                      # for more than one epoch (that is, two epochs)
                      patience=1,
                      ),
                  # Saves the current weights after every epoch
                  keras.callbacks.ModelCheckpoint(
                      # Path to the destination model file
                      filepath='my_model.h5',
                      # These two arguments mean you won’t overwrite the
                      # model file unless val_loss has improved, which allows
                      # you to keep the best model seen during training.
                      monitor='val_loss',
                      save_best_only=True,
                      )
                  ]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              # You monitor accuracy, so it should
              # be part of the model’s metrics.
              metrics=['acc'])

# Note that because the callback will monitor validation loss and
# validation accuracy, you need to pass validation_data to the call to fit.
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

####THE REDUCELRONPLATEAU CALLBACK
You can use this callback to reduce the learning rate when the validation loss has stopped improving. Reducing or increasing the learning rate in case of a loss plateau is is an effective strategy to get out of local minima during training. The following example uses the ReduceLROnPlateau callback: 

In [0]:
callbacks_list = [
                  keras.callbacks.ReduceLROnPlateau(
                      # Monitors the model’s validation loss
                      monitor='val_loss',
                      # Divides the learning rate by 10 when triggered
                      factor=0.1,
                      # The callback is triggered after the validation
                      # loss has stopped improving for 10 epochs
                      patience=10,
                      )
                  ]
# Because the callback will monitor the validation loss, you
# need to pass validation_data to the call to fit.                  
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

#### WRITING YOUR OWN CALLBACK
If you need to take a specific action during training that isn’t covered by one of the built-in callbacks, you can write your own callback. Callbacks are implemented by subclassing the class keras.callbacks.Callback. You can then implement any number of the following transparently named methods, which are called at various points during training:

* on_epoch_begin - Called at the start of every epoch
* on_epoch_end - Called at the end of every epoch

* on_batch_begin - Called right before processing each batch
* on_batch_end - Called right after processing each batch

* on_train_begin - Called at the start of training
* on_train_end - Called at the end of training

These methods all are called with a logs argument, which is a dictionary containing information about the previous batch, epoch, or training run: training and validation metrics, and so on. Additionally, the callback has access to the following attributes:
* self.model—The model instance from which the callback is being called
* self.validation_data—The value of what was passed to fit as validation data

Here’s a simple example of a custom callback that saves to disk (as Numpy arrays) the activations of every layer of the model at the end of every epoch, computed on the first sample of the validation set: 


In [0]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        # Called by the parent model before training, to inform
        # the callback of what model will be calling it
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        # Model instance that returns the activations of every layer
        self.activations_model = keras.models.Model(model.input, layer_outputs)
    
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        # Obtains the first input sample of the validation data
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        with open('activations_at_epoch_' + str(epoch) + '.npz', 'w') as f:
            np.savez(f, activations)
        

Using TensorFlow backend.


This is all you need to know about callbacks—the rest is technical details, which you can easily look up. Now you’re equipped to perform any sort of logging or preprogrammed intervention on a Keras model during training.

###7.2.2 Introduction to TensorBoard: the TensorFlow visualization framework
To do good research or develop good models, you need rich, frequent feedback about what’s going on inside your models during your experiments. That’s the point of running experiments: to get information about how well a model performs—as much information as possible. Making progress is an iterative process, or loop: you start with an idea and express it as an experiment, attempting to validate or invalidate your idea. You run this experiment and process the information it generates. This inspires your next idea. The more iterations of this loop you’re able to run, the more refined and powerful your ideas become. Keras helps you go from idea to experiment in the least possible time, and fast GPUs can help you get from experiment to result as quickly as possible. But what about processing the experiment results? That’s where TensorBoard comes in.

This section introduces TensorBoard, a browser-based visualization tool that comes packaged with TensorFlow. Note that it’s only available for Keras models when you’re using Keras with the TensorFlow backend.

The key purpose of TensorBoard is to help you visually monitor everything that goes on inside your model during training. If you’re monitoring more information than just the model’s final loss, you can develop a clearer vision of what the model does and doesn’t do, and you can make progress more quickly. TensorBoard gives you access to several neat features, all in your browser:
* Visually monitoring metrics during training
* Visualizing your model architecture
* Visualizing histograms of activations and gradients
* Exploring embeddings in 3D
Let’s demonstrate these features on a simple example. You’ll train a 1D convnet on the IMDB sentiment-analysis task. You’ll consider only the top 2,000 words in the IMDB vocabulary, to make visualizing word embeddings more tractable. 

In [1]:
# Text-classification model to use with TensorBoard
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,input_length=max_len,name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-tra

Before you start using TensorBoard, you need to create a directory where you’ll store the log files it generates.
```
$ mkdir my_log_dir
```
Let’s launch the training with a TensorBoard callback instance. This callback will write log events to disk at the specified location.

In [0]:
!mkdir my_log_dir

In [3]:
# Training the model with a TensorBoard callback
callbacks = [
             keras.callbacks.TensorBoard(
                 # Log files will be written at this location
                 log_dir='my_log_dir',
                 # Records activation histograms every 1 epoch
                 histogram_freq=1,
                 # Records embedding data every 1 epoch
                 embeddings_freq=1,
                 )
             ]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

/usr/local/lib/python3.6/dist-packages/keras/callbacks/tensorboard_v2.py:102: UserWarning: The TensorBoard callback does not support embeddings display when using TensorFlow 2.0. Embeddings-related arguments are ignored.
  warnings.warn('The TensorBoard callback does not support '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 68s 3ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 2/20
20000/20000 [==============================] - 67s 3ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 3/20
20000/20000 [==============================] - 68s 3ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 4/20
20000/20000 [==============================] - 68s 3ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 5/20
20000/20000 [==============================] - 68s 3ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 6/20
20000/20000 [==============================] - 68s 3ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 7/20
20000/20000 [==============================] - 67s 3ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc:

At this point, you can launch the TensorBoard server from the command line, instructing it to read the logs the callback is currently writing. The tensorboard utility should have been automatically installed on your machine the moment you installed TensorFlow (for example, via pip):
```
$ tensorboard --logdir=my_log_dir
```
You can then browse to http://localhost:6006 and look at your model training. In addition to live graphs of the training and validation metrics, you get access to the Histograms tab, where you can find pretty visualizations of histograms of activation values taken by your layers.


#### Tensorboard for Google Colab


In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
%tensorboard --logdir .\my_log_dir